In [1]:
using LinearAlgebra;
using Random;
Random.seed!(42);

In [2]:
macro prettyshow(var)
    quote
        $(print(string(var), " is "))
        $(show(stdout, MIME"text/plain"(), eval(var)))
        $(println())
    end
end

@prettyshow (macro with 1 method)

## 线性变换与基

矩阵表示一个线性变换，更具体的说，是在一个线性变换**一组特定基下**的表示。也就是说，同一个线性变换，换了一组基，矩阵的样子就变了。但是，它内在的线性变换还是同一个，所以这些不同基下的表示矩阵存在关联，这种关联被称作**相似**。

教科书上的定义说，两个 $n$ 阶方阵 $A$、$B$ 相似是指，存在 $n$ 阶可逆矩阵 $S$，满足：
$$
    B = S^{-1} A S
$$

我们可以把 $S$ 看作把当前坐标从 $B$ 使用的基变换成 $A$ 使用的基的一个过渡矩阵，那么 $S^{-1}$ 就是把当前坐标从 $A$ 使用的基变换回 $B$ 使用的基的过渡矩阵。记 $A$、$B$ 对应的线性变换为 $T$，那么 上式可以解释为：$B$ 矩阵的作用是，**先从 $B$ 的基换到 $A$ 的基，在该基下作完变换 $T$ 后，在换回 $B$ 的基**。

举个例子来辅助理解，把线性变换比作对句子的一个“润色”操作，$A$、$B$ 使用的基比作中文和英文，$A$、$B$ 矩阵分别在中文和英文下完成同一个操作（润色）。那么，对英文句子润色的操作（$B$ 矩阵）可以分解成如下步骤：

1. 翻译成中文（从 $B$ 的基换到 $A$ 的基）；
2. 在中文下完成润色操作（乘以 $A$）；
3. 翻译回英文（从 $A$ 的基换回 $B$ 的基）；

下面我们用具体例子感受下上面所描述的过程：

In [3]:
n = 4
B = mod.(rand(Int, n, n), 10)
x = mod.(rand(Int, n), 10)
@prettyshow B
@prettyshow x

B is 4×4 Array{Int64,2}:
 2  8  1  4
 8  3  9  0
 6  5  4  4
 5  3  9  2
x is 4-element Array{Int64,1}:
 7
 5
 3
 0


假定 $B$ 使用的基是标准正交基，且通过如下 $S$ 矩阵把 $B$ 的基换到 $A$ 的基，那么 $A$ 矩阵可以写作：$A = S B S^{-1}$。

In [4]:
S = ones(Int, n, n)
triu!(S)
@prettyshow S
S_inv = inv(S)
S_inv = convert.(Int, S_inv)
@prettyshow S_inv

A = S * B * S_inv
A = convert.(Int, A)
@prettyshow B

S is 4×4 Array{Int64,2}:
 1  1  1  1
 0  1  1  1
 0  0  1  1
 0  0  0  1
S_inv is 4×4 Array{Int64,2}:
 1  -1   0   0
 0   1  -1   0
 0   0   1  -1
 0   0   0   1
B is 4×4 Array{Int64,2}:
 2  8  1  4
 8  3  9  0
 6  5  4  4
 5  3  9  2


我们来检查一下，$B$ 矩阵是否能分解成我们所说的那样？

In [5]:
# 直接使用 B 作变换
y = B * x

# 1. 变换成 $A$ 的基
tmp = S * x
# 2. 在 $A$ 的基下完成线性变换
tmp = A * tmp
# 3. 变换回 $B$ 的基
y2 = S_inv * tmp

@prettyshow y
@prettyshow y2

y is 4-element Array{Int64,1}:
 57
 98
 79
 77
y2 is 4-element Array{Int64,1}:
 57
 98
 79
 77


可以看到，两种方法计算出来的结果是一致的，这验证了我们对公式的解释。

通过上面的过程，我们对相似的理解已经不仅限于记住公式，更理解了其中每一个矩阵的含义与作用。

## 对角化

假设 $A$ 相似于一个对角阵 $\Lambda$，写成公式就是 $\Lambda = S^{-1} A S$，这个过程被称作对角化。

为什么需要对角化？首先，它在矩阵的幂运算中很有用，因为：
$$
    A^{n} = (S \Lambda S^{-1})^{n} = (S \Lambda S^{-1}) \cdot (S \Lambda S^{-1})\cdot \cdots \cdot (S \Lambda S^{-1}) = S \Lambda^{n} S^{-1}
$$

通过以上过程，我们把 $A$ 的幂运算转化为了对角阵的幂运算，大大减小了计算量。

其次，从线性变换的角度看，对角化的过程实质上是**为 $A$ 表示的线性变换 $T$ 找到了一组基 $\mathcal{B}$，$T$ 在这组基下的矩阵表达很简单**，就是一个对角阵。也就是说，对于 $\mathcal{B}$ 中的每一个基，$A$ 矩阵的作用实质上只是乘了一个缩放系数。不仅计算上方便，更有助于我们理解 $T$ 的本质。

记 $\Lambda$ 对角线上第 $i$ 个元素为 $\lambda_i$，$\mathcal{B}$ 中的基为 $v_1, v_2, \dotsc, v_n$，我们有：
$$
    A v_i = \lambda_i v_i \quad i=1,2,\dotsc,n
$$
上式即矩阵 $A$ 特征值和特征向量的定义，$\lambda_i$ 为特征值，$v_i$ 为特征向量。所以，对角化结果中的 $\Lambda$ 即矩阵 $A$ 的特征值，$S$ 中的列即矩阵 $A$ 的特征向量。

In [6]:
λ, S = eigen(A);

A2 = S * Diagonal(λ) * inv(S)
A2 = round.(Int, A2)
@prettyshow A
@prettyshow A2

A is 4×4 Array{Int64,2}:
 21  -2   4  -13
 19  -8  11  -16
 11  -3   5   -7
  5  -2   6   -7
A2 is 4×4 Array{Int64,2}:
 21  -2   4  -13
 19  -8  11  -16
 11  -3   5   -7
  5  -2   6   -7


容易验证，相似关系是一个等价关系（满足自反、对称、可传递）。所以如果 $A$、$B$ 相似，且 $A$ 相似于对角阵 $\Lambda$，那么 $B$ 也相似于 $\Lambda$。

所以，$A$ 与 $B$ 矩阵都可以找到一组基，使得在这两组基下的矩阵表示很简单。**基的选取不影响线性变换的本质，线性变换的本质在于中间的 $\Lambda$**（可对角化的前提下）。

通过对角化和相似关系，我们得以窥探不同矩阵外衣下，同一个线性变换的本质。研究对角化后的矩阵 $\Lambda$ 的性质，相当于研究了其在任意基下表示矩阵的性质，这大大加深了我们对矩阵的理解。